In [32]:
from keras.layers import Conv2D, Dense, Input, Flatten, Dropout, MaxPooling2D, Activation, concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Model
from matplotlib.pyplot import imshow
import keras.preprocessing as kp
from matplotlib import pyplot as plt
import pickle
import numpy as np
%matplotlib inline


In [2]:
!git clone -l -s https://github.com/monacotime/DeepForSpeed.git DeepForSpeed

Cloning into 'DeepForSpeed'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 2097 (delta 2), reused 6 (delta 1), pack-reused 2089
Receiving objects: 100% (2097/2097), 1.82 GiB | 31.42 MiB/s, done.
Resolving deltas: 100% (39/39), done.
Checking out files: 100% (2010/2010), done.


**class_mode:** One of "categorical", "binary", "sparse", "input", or None. Default: "categorical". Determines the type of label arrays that are returned: - "categorical" will be 2D one-hot encoded labels, - "binary" will be 1D binary labels, "sparse" will be 1D integer labels, - "input" will be images identical to input images (mainly used to work with autoencoders). - If ***`None`***, ***no labels are returned (the generator will only yield batches of image data, which is useful to use with model.predict_generator()). Please note that in case of class_mode None, the data still needs to reside in a subdirectory of directory for it to work correctly.***

`This block is not true but im keeping it as a reminder`


> What happens is that if class = none, all the images are mixed and no class data is there but if the class mode is categorical the all data are still mixed and jumbled but the class data is there so regardless you cant index the two types of related images from 2 different direcotory together. that is why you have got to make 2 generaters and they will give out output in a sequence which can be later mixed to make the final related input.

You done need to structure it like how I did:
## I now realize that flowfromdir(class = None) 

[0][0] -> first class first image (no class data)

[1][0] -> second class first image (no class data)

In [33]:
batch_size = 8
before = r"/content/DeepForSpeed/Dataset/first"
after = r"/content/DeepForSpeed/Dataset/second"
img_height,img_width = 500,500
input_shape = (500,500,1)
trainsetsize = 800
testsetsize = 200
with open(r"/content/DeepForSpeed/speed.txt", "rb") as fp:
    speed = pickle.load(fp)

In [34]:
def generate_generator_multiple(dir1, dir2, batch_size, img_height,img_width, subset):
    dataset1 = kp.image_dataset_from_directory(
                                            dir1,
                                            labels = speed,
                                            label_mode= "int",
                                            class_names=None,
                                            color_mode="grayscale",
                                            batch_size=batch_size,
                                            image_size=(img_height,img_width),
                                            shuffle=False,
                                            seed=1,
                                            validation_split=0.2,
                                            subset=subset,
                                            interpolation="bilinear",
                                            follow_links=False,
                                            ).repeat()
    dataset2 = kp.image_dataset_from_directory(
                                            dir2,
                                            labels = speed,
                                            label_mode= "int",
                                            class_names=None,
                                            color_mode="grayscale",
                                            batch_size=batch_size,
                                            image_size=(img_height,img_width),
                                            shuffle=False,
                                            seed=1,
                                            validation_split=0.2,
                                            subset=subset,
                                            interpolation="bilinear",
                                            follow_links=False,
                                            ).repeat()
    it1 = iter(dataset1)
    it2 = iter(dataset2)
    
    while True:
            X1 = next(it1)
            X2 = next(it2)
            yield [X1[0]/255, X2[0]/255], X1[1] #Yield both images and their mutual label 

In [35]:
traingenerator=generate_generator_multiple(dir1=before,
                                           dir2=after,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_height,
                                           subset = "training")
testgenerator=generate_generator_multiple(dir1=before,
                                           dir2=after,
                                           batch_size=batch_size,
                                           img_height=img_height,
                                           img_width=img_height,
                                           subset = "testing")

In [44]:
def create_convolution_layers(input_img):
    model = Conv2D(32, (3, 3), padding='same', input_shape=input_shape)(input_img)
    model = Activation("relu")(model)
    model = MaxPooling2D((2, 2),padding='same')(model)
    model = Dropout(0.25)(model)

    return model

In [45]:
first_input = Input(shape = input_shape, name='input_1')
before_model = create_convolution_layers(first_input)

second_input = Input(shape = input_shape, name='input_2')
after_model = create_convolution_layers(second_input)

conc = concatenate([before_model,after_model])
conc = Flatten()(conc)

dense = Dense(64, activation="relu")(conc)
dense = Dropout(0.25)(dense)

dense = Dense(32, activation="relu")(dense)
dense = Dropout(0.25)(dense)

output = Dense(1, activation = "relu")(dense)

model = Model(inputs = [first_input, second_input], outputs = [output])

opt = optimizers.Adam()
model.compile(loss = "mean_squared_error",
              optimizer = opt,
              metrics = ['acc'])
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 500, 500, 1) 0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 500, 500, 32) 320         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 500, 500, 32) 320         input_2[0][0]                    
_______________________________________________________________________________________

In [40]:
class new_callback(tf.keras.callbacks.Callback):
    def epoch_end(self, epoch, logs={}): 
        if(logs.get('accuracy')> 0.90): # select the accuracy
            print("\n !!! 90% accuracy, no further training !!!")
            self.model.stop_training = True

callbacks = new_callback()

In [ ]:
history = model.fit(traingenerator, 
                    steps_per_epoch=100, 
                    epochs = 200, 
                    verbose =1, 
                    callbacks= callbacks)

In [ ]:
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.save("model.h5")